In [ ]:
import glob
import numpy as np
import cv2 as cv
import json
import pandas as pd
from tqdm import tqdm


import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

import geojson

In [ ]:
def IoU(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersect = np.sum(y_true * y_pred)
    fsum = np.sum(y_true)
    ssum = np.sum(y_pred)
    dice = (2 * intersect ) / (fsum + ssum)
    return dice

In [ ]:
def get_mask_index(feature, classes):
    class_type = feature['properties']['classification']['name']

    for idx, name in enumerate(classes):
        if class_type.lower() == name.lower():
            return idx

    # else return Other cells
    return 0


def get_mask(shape, annotations, classes):
    x, y = int(shape[0]), int(shape[1])

    classes_masks = [
        np.zeros((x, y, 1), dtype='uint8')
        for _ in range(len(classes))
    ]

    for feat in annotations:
        geometry_name = 'geometry'
        coors = feat[geometry_name]['coordinates'][0]
        try:
            pts = [[round(c[0]), round(c[1])] for c in coors]
        except:
            pts = [[round(c[0]), round(c[1])] for c in coors[0]]
        cv.fillPoly(
            classes_masks[get_mask_index(feat, classes)],
            [np.array(pts)],
            1
        )  # fill with ones if cell present

    mask = np.concatenate(classes_masks, axis=2)
    return mask

In [ ]:
def get_metrics(df, ground_truth_path, predicted_path, classes, shape, name, distance, area, only_immune, num_cells):
    data = {
        'name': name,
        'min_distance': distance,
        'min_area': area,
        'only_immune': only_immune,
        'num_cells': num_cells
    }
    gj = geojson.load(open(ground_truth_path))
    ground_truth = get_mask(shape, gj['features'], classes)
    
    gj = geojson.load(open(predicted_path))
    predicted = get_mask(shape, gj['features'], classes)
    
    for c_idx, c in enumerate(classes):
        data[f'IoU {c}'] = IoU(ground_truth[:, :, c_idx], predicted[:, :, c_idx])
        data[f'Dice {c}'] = dice_coef(ground_truth[:, :, c_idx], predicted[:, :, c_idx])
    
    return pd.concat([
            df,
            pd.DataFrame([data])
        ])

In [ ]:
def calculate_metrics():
    df = pd.DataFrame()
    
    experiment_path = r'D:\DBSCAN'
    experiments = glob.glob(f'{experiment_path}\\*')
    
    ground_truth_path = r'D:\Master Thesis\Data\Annotations'
    
    for experiment in experiments:
        predicted_path = experiment
        num_cells = experiment.replace('\\', '/').split('/').pop().split('-')[0]
        distance = experiment.replace('\\', '/').split('/').pop().split('-')[1]
        area = experiment.replace('\\', '/').split('/').pop().split('-')[2]
        only_immune = experiment.replace('\\', '/').split('/').pop().split('-')[3]
        print(distance, area, only_immune)
        
        classes = ['Inflammation'] 

        json_path = './data/images_512.json'

        with open(json_path) as json_file:
            json_data = json.load(json_file)

        ground_truth_files = glob.glob(f'{ground_truth_path}\\*.geojson')
        predicted_files = glob.glob(f'{predicted_path}\\*.geojson')

        for image in tqdm(json_data['images'], total=len(json_data['images'])):
            name = image['name']
            predicted = [geo for geo in predicted_files if name in geo]

            ground_truth = [geo for geo in ground_truth_files if name in geo]
            if len(ground_truth) == 0 or len(predicted) == 0:
                continue
                
            predicted = predicted[0]
            ground_truth = ground_truth[0]
            shape = (image['height'], image['width'])

            df = get_metrics(df, ground_truth, predicted, classes, shape, name, distance, area, only_immune, num_cells)

    return df

In [ ]:
df = calculate_metrics()

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
num_cells = [10, 15, 20]
areas = [15_000, 20_000, 25_000, 30_000, 35_000, 40_000]
distances = [40, 45, 50, 55, 65, 75, 80, 90, 100]
only_immunes = [False, True]

In [ ]:
data = [
    {
        'Maximálna vzdialenosť': distance,
        'Minimána plocha': area,
        'Iba imunitné': only_immune,
        'Počet buniek': num_cell,
        'IoU [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['IoU Inflammation'].mean() * 100,
        'Min IoU [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['IoU Inflammation'].min() * 100,
        'Max IoU [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['IoU Inflammation'].max() * 100,
        'Dice [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['Dice Inflammation'].mean() * 100,
        'Min Dice [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['Dice Inflammation'].min() * 100,
        'Max Dice [%]': df[(df['IoU Inflammation'] != 0) & (df['min_distance'].astype(str) == f'{distance}') & (df['min_area'].astype(str) == f'{area}') & (df['only_immune'].astype(str) == f'{only_immune}') & (df['num_cells'].astype(str) == f'{num_cell}')]['Dice Inflammation'].max() * 100,
    }
    for area in areas
    for distance in distances
    for num_cell in num_cells
    for only_immune in only_immunes
]

In [ ]:
data

In [ ]:
metrics_df = pd.DataFrame(data)

In [ ]:
print(metrics_df.sort_values(by=['Iba imunitné', 'Maximálna vzdialenosť', 'Minimána plocha']).round(2).to_latex(index=False))

In [ ]:
metrics_df.sort_values(by=['IoU [%]'], ascending=False)